# Talk Page Edit Schema QA
[TASK](https://phabricator.wikimedia.org/T286076)

[Add distinct event to talk_page_edit for when a new section is added](https://phabricator.wikimedia.org/T301496)

# Upload Data

In [7]:
shhh <- function(expr) suppressPackageStartupMessages(suppressWarnings(suppressMessages(expr)))
shhh({
    library(tidyverse); library(glue); library(lubridate); library(scales)
})

In [1]:
# Collect talk page edit events

query <-
"SELECT
 component_type,
 topic_id,
 comment_parent_id,
comment_id,
page_namespace,
performer.user_id As `user`,
performer.user_is_anonymous As is_anon,
revision_id,
session_id,
integration,
action,
performer.user_edit_count As edit_count,
performer.user_edit_count_bucket As edit_count_bucket,
`database`,
meta.dt
FROM
event.mediawiki_talk_page_edit
WHERE
-- since deployment of the patch to add heading component type
    year = 2022 
    AND month = 02
    AND day >=18
"

In [2]:
collect_talk_events <-  wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [5]:
collect_talk_events$dt <- as.Date(collect_talk_events$dt, format = "%Y-%m-%d")

In [8]:
daily_events <-  collect_talk_events %>%
    group_by(dt) %>%
    count() %>%
    arrange(dt)

daily_events

dt,n
<date>,<int>
2022-02-18,4241
2022-02-19,3808
2022-02-20,3832
2022-02-21,4464
2022-02-22,2573


We start recording events on November 3rd, which is when the change was deployed. 

In [9]:
talk_events_total <- collect_talk_events %>%
    summarise(total_events = n())

talk_events_total

total_events
<int>
18918


# Count Distinct Logged-In Users

In [10]:
# Number of logged in users
talk_users_unique <- collect_talk_events %>%
    filter(is_anon == 'false') %>%
    summarise(unique_users = n_distinct(user))

talk_users_unique

unique_users
<int>
5533


# Logged Out User Check

In [11]:
talk_users_anon <- collect_talk_events %>%
    filter(is_anon == 'true'| user == 0 ) %>%
   summarise( n_events = n())

talk_users_anon

n_events
<int>
834


We are now logging logged out events

# User Edit Count Check

In [12]:
talk_events_byeditcount <- collect_talk_events %>%
    group_by(edit_count_bucket) %>%
    summarise(total_events = n())

talk_events_byeditcount

`summarise()` ungrouping output (override with `.groups` argument)



edit_count_bucket,total_events
<chr>,<int>
1-4 edits,423
100-999 edits,2048
1000+ edits,13919
5-99 edits,1694
N/A,834


ISSUE: We do not appear to be logging any edits in the 1-4 edit bucket group.
UPDATE: We are now logging edit count bucket events correctly. All logged out users are assigned the 'NA' value.

# Integration Check

In [13]:
talk_events_byintergration <- collect_talk_events %>%
    group_by(integration) %>%
    summarise(total_events = n())

talk_events_byintergration

`summarise()` ungrouping output (override with `.groups` argument)



integration,total_events
<chr>,<int>
discussiontools,15070
page,3848


We've recorded talk page events using discussion tools and on the page. 

# Cross Wiki Check

In [14]:
talk_events_bywiki <- collect_talk_events %>%
    group_by(database) %>%
    summarise(total_events = n())

talk_events_bywiki

`summarise()` ungrouping output (override with `.groups` argument)



database,total_events
<chr>,<int>
afwiki,9
alswiki,1
arwiki,293
arzwiki,7
atjwiki,1
azbwiki,2
azwiki,68
bclwiki,1
bewikisource,1


# Component Types

In [15]:
talk_events_bycomponent <- collect_talk_events %>%
    group_by(component_type) %>%
    summarise(total_events = n())

talk_events_bycomponent

`summarise()` ungrouping output (override with `.groups` argument)



component_type,total_events
<chr>,<int>
comment,852
response,13298
topic,4768


There are more responses than comments or topics as expected.

There are more topics than comments - is this expected. I think so as thre might be more people that start new discussions than those that post high-level comments.

# Action

In [16]:
talk_events_byaction <- collect_talk_events %>%
    group_by(action) %>%
    summarise(total_events = n())

talk_events_byaction

`summarise()` ungrouping output (override with `.groups` argument)



action,total_events
<chr>,<int>
publish,18918


In [ ]:
All recorded events have been logged as publish events as expected.

# Namespace

In [17]:
talk_events_bynamespace <- collect_talk_events %>%
    group_by(page_namespace) %>%
    summarise(total_events = n()) %>%
    arrange(desc(total_events))

talk_events_bynamespace

`summarise()` ungrouping output (override with `.groups` argument)



page_namespace,total_events
<int>,<int>
3,8092
4,4515
1,4383
5,1077
11,250
103,232
12,52
101,45
119,44


The most events have occured on User Talk pages (38.9%), followed by project talk (27%) and then article talk pages (24%). There are some events that occur on non talk pages. 364 events were recored on article (main) namespace pages. - Do we know what might cause this.

# Comment and Response IDs

Per current set-up, `comment_id` will change with every event. For the very first comment in a thread that'll be the ID of the topic (but the component_type will be `comment` rather than `response` and `comment_id` will equal `topic_id`, so this is easy to distinguish).

In [40]:
 #find any instances where the comment id is equal to the comment parent id

comment_id_check <- collect_talk_events %>%
    filter(collect_talk_events$comment_id 
           ==  collect_talk_events$comment_parent_id)

comment_id_check

component_type,topic_id,comment_parent_id,comment_id,page_namespace,user,is_anon,revision_id,session_id,integration,action,edit_count,edit_count_bucket,database,dt
<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<date>


In [ ]:
No instances where the comment_id is equal to comment_parent_id as expected

In [ ]:
# find top_level comments where the comment id is equal to the topic id. Should all be lableld comment

top_level_commment <- collect_talk_events %>%
    filter(collect_talk_events$comment_parent_id 
           ==  collect_talk_events$topic_id & component_type == 'comment')

top_level_commment 

There are several top level comments, indicated by a comment_parent_id that matches a topic_id. All of these are labeled as comments as expected. Next we'll check that all responses are labled correctly. 

In [19]:
# make sure responses are labeled correctly

response_check <- collect_talk_events %>%
    filter(collect_talk_events$comment_parent_id 
           ==  collect_talk_events$topic_id & component_type == 'response')
response_check

component_type,topic_id,comment_parent_id,comment_id,page_namespace,user,is_anon,revision_id,session_id,integration,action,edit_count,edit_count_bucket,database,dt
<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<date>


In [ ]:
Confirmed there are no instances where a response event has the same comment_parent_id and topic_id.

In [ ]:
non_top_level_comments <- collect_talk_events %>%
    filter(collect_talk_events$comment_parent_id 
           !=  collect_talk_events$topic_id & component_type == 'comment') %>%
    summa

non_top_level_comments

There are component_type = comment events where the comment_parent_id does not match the topic_id. Is this expected? What's the difference between this and a response?

# Topic ID Check

In [ ]:
# review raw dataset of component_type = topic to see how associated ids are recorded
new_topics <- collect_talk_events %>%
    filter(component_type == 'topic')

new_topics 

In [25]:
 #check for any instaances where topic id does not equal parent id
topic_check_id <- collect_talk_events %>%
  filter(collect_talk_events$comment_parent_id 
           !=  collect_talk_events$topic_id & component_type == 'topic')

topic_check_id 

component_type,topic_id,comment_parent_id,comment_id,page_namespace,user,is_anon,revision_id,session_id,integration,action,edit_count,edit_count_bucket,database,dt
<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<date>


In [ ]:
Confirmed there are no new topics where the comment parent id does not equal the topic id as expected.

In [27]:
 #check for any instrances where topic id and comment id equal each other
topic_check_id_2 <- collect_talk_events %>%
  filter(collect_talk_events$comment_id 
           ==  collect_talk_events$topic_id & component_type == 'topic')

topic_check_id_2

component_type,topic_id,comment_parent_id,comment_id,page_namespace,user,is_anon,revision_id,session_id,integration,action,edit_count,edit_count_bucket,database,dt
<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<date>


In [ ]:
Confirmed there are no topic ids that equal the comment id as expected

In [ ]:
# Check that only one event is sent for each topic event

In [ ]:
topic_check_same_comment <- collect_talk_events %>%
    filter(topic_id == "h-Disambiguating_the_unambiguous-2022-02-18T00:19:00.000Z")

topic_check_same_comment

# KPI Test

Check that I can perform the joins necessary to calculate the KPIs identifed for this data:
- For all comments and new topics with a response, the average time duration from a contributor posting on a talk page to them receiving a response from a Junior Contributor by post type (e.g. comment and new topics)

In [ ]:
If there's a response, then it should be listed as a parent_id
- You can find comments and topics without a response by looking for new comment_ids that are not also labeled as comment_parent_ids


In [35]:
comments_with_response <- collect_talk_events  %>%
    filter(comment_id %in% comment_parent_id) # confirms that the comment recieved a response at some point


Steps would be to find all sessions 
- Find all sessions where comment_id is in the comment_parent_id list. These are all topics or new comments that have received a response. FEASIBLE
- Select the user_id and dt and comment_parent_id and timestamp for all these comments. FEASIBLE
- Find sessions events where comment_id is equal to the comment_parent_id. These are all comments that have received a response. FEASBILE: JOIN Comment_parent_id on comment_id
- Find the user_id and timestamp for these comments. FEASIBLE
- Final Query is to subtract the two timestamps FEASIBLE

In [ ]:
# Check to see if you can find comment_parent_id in the comment_id list

In [36]:
comments_without_response <- collect_talk_events  %>%
    filter(comment_parent_id %in% comment_id) # confirms that the comment recieved a response at some point



# Check that the timestamp for the initial comment occurs before the response

In [ ]:
initial_comment_data <- collect_talk_events  %>%
    filter(comment_parent_id == 'c-Barter84-2021-11-09T16:17:00.000Z-Djhuty-2021-11-08T15:14:00.000Z') 

initial_comment_data

In [ ]:
response_comment_data <- collect_talk_events  %>%
    filter(comment_id == 'c-Barter84-2021-11-09T16:17:00.000Z-Djhuty-2021-11-08T15:14:00.000Z') 

response_comment_data

In [ ]:
# Check if there are duplicate comment_id events
 duplicate_comments <- collect_talk_events %>%
     group_by(comment_id) %>% 
      filter(n()>1)
 

Potential Issues: There's s number of events with the same comment_id logged. Different revision and sessions ids. Why would this happen?

# Check joins to EditAttemptStep

In [25]:
query <-

"SELECT
  tpe.session_id,
  eas.event.editing_session_id,
  tpe.performer.user_id,
  eas.event.user_id
FROM
  event.mediawiki_talk_page_edit tpe
LEFT JOIN 
  event.editattemptstep eas
  ON session_id = eas.event.editing_session_id
  AND eas.year = 2021 and eas.MOnth >= 11
  WHERE
  tpe.year = 2021
  and tpe.month = 11
  AND tpe.integration = 'discussiontools'

"

In [26]:
collect_talk_events_weditor  <-  wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [ ]:
collect_talk_events_weditor %>%
    filter(editing_session_id == 'NULL')

There's some sessions in editattemptstep but not all. All discussiontools related sessions are in editattemptstep but not all page related sessions. This is likely due to sampling differences. Need to see if we can match sampling rates. That's where we get platform data so we'll need to apply the same sampling rates as possible. 